In [1]:
import pymongo
import pprint
from pymongo import MongoClient
import ete3
from ete3 import NCBITaxa
import pandas as pd
ncbi = NCBITaxa()
client = MongoClient()
db = client.COCONUT2020
collection = db.curatedNP

In [2]:
data = collection.find({})
tables = pd.DataFrame(list(data))

In [3]:
# Obtain a diccionary with the coconut_id as key and the ranks of interest as value
dicc = {}
for i in range(0, len(tables)):
    row = tables.loc[i,'tax_classification']
    ID = tables.loc[i, '_id']
    dicc_lineage = {}
    n = 0
    while n < len(row):
        for elem in row:
            value = list(elem.values())
            value1 = value[0]
            for j in value1:
                x = j['taxid']
                lineage = ncbi.get_lineage(x)
                add = []
                for i in lineage:
                    rank = ncbi.get_rank([i])
                    values = list(rank.values())
                    if 'superkingdom' in values:
                        add.append(i)
                    if 'kingdom' in values:
                        add.append(i)
                    if 'phylum' in values:
                        add.append(i)
                    if 'class' in values:
                        add.append(i)
                    if 'order' in values:
                        add.append(i)
                    if 'family' in values:
                        add.append(i)
                    if 'genus' in values:
                        add.append(i)
                    if 'species' in values:
                        add.append(i)
                dicc_lineage[n] = add
            n += 1
    dicc[ID] = dicc_lineage

c:\users\marta\appdata\local\programs\python\python38-32\lib\site-packages\ete3\ncbi_taxonomy\ncbiquery.py:243: UserWarning: taxid 86706 was translated into 2706154
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
c:\users\marta\appdata\local\programs\python\python38-32\lib\site-packages\ete3\ncbi_taxonomy\ncbiquery.py:243: UserWarning: taxid 46399 was translated into 228393
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
c:\users\marta\appdata\local\programs\python\python38-32\lib\site-packages\ete3\ncbi_taxonomy\ncbiquery.py:243: UserWarning: taxid 259302 was translated into 206448
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
c:\users\marta\appdata\local\programs\python\python38-32\lib\site-packages\ete3\ncbi_taxonomy\ncbiquery.py:243: UserWarning: taxid 285843 was translated into 2769829
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid

In [ ]:
# Completing the db
column_names = ['superkingdom', 'kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
dicc_lin = {}
for key in dicc:
    list_data = {}
    list_data['lineage'] = []
    lineages = dicc[key]
    for k in lineages:
        x = lineages[k]
        n = 0
        data = {}
        while n < len(x):
            data[column_names[n]] = x[n]
            n += 1
        list_data['lineage'].append(data)
    dicc_lin[key] = list_data

for key in dicc_lin:
    x = dicc_lin[key]
    collection.update_one({'_id': key}, {'$set': x})